In [1]:
import psycopg2
import re
import psycopg2.extras
import pickle
import math

# NYC RESTAURANT INSPECTIONS

The city of New York does restaurant inspections and assigns a grade. I had a folder of inspections data from the last four years in the form of csv formatted text files. I cleaned the data, and loaded it into five Postgresql tables -- actions, cuisines, violations, grades, and boroughs -- that I then used for further analysis.

Two notes:

(1) Some files contained malformatted text.  I used the 'iconv' unix utility to convert to utf-8 encoding.

(2) I also had to reparse files to get rid of commas (',') within a single field (which would've tricked csv parsers into breaking up the field).

## CALCULATING MEAN GRADE BY ZIPCODE

In this section, I extracted the mean grade, standard error, and number of inspections for each of the 183 zipcodes in NYC with over 100 inspections.

In [131]:
with open('WebExtract.txt') as f:
    content = f.readlines()
for lines in content[0:4]:
    print(lines)

"CAMIS","DBA","BORO","BUILDING","STREET","ZIPCODE","PHONE","CUISINECODE","INSPDATE","ACTION","VIOLCODE","SCORE","CURRENTGRADE","GRADEDATE","RECORDDATE"

"30075445","MORRIS PARK BAKE SHOP","2","1007      ","MORRIS PARK AVE                                   ","10462","7188924968","08","2014-03-03 00:00:00","D","10F","2","A","2014-03-03 00:00:00","2014-09-04 06:01:28.403000000"

"30112340","WENDY'S","3","469","FLATBUSH AVENUE","11225","7182875005","39","2014-07-01 00:00:00","F","06A","23","B","2014-07-01 00:00:00","2014-09-04 06:01:28.403000000"

"30191841","DJ REYNOLDS PUB AND RESTAURANT","1","351","WEST 57 STREET","10019","2122452912","03","2013-07-22 00:00:00","D","10B","11","A","2013-07-22 00:00:00","2014-09-04 06:01:28.403000000"



In [552]:
conn = psycopg2.connect("dbname='vagrant' user='vagrant'")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS grades")
cur.execute("CREATE TABLE grades (CAMIS INT, DBA TEXT, BORO INT, BUILDING TEXT, STREET TEXT, ZIPCODE INT, PHONE varchar(13), CUISINECODE INT, INSPDATE timestamp, ACTION varchar(1), VIOLCODE varchar(3), SCORE INT, CURRENTGRADE varchar(1), GRADEDATE varchar(30), RECORDDATE timestamp)")
conn.commit()

In [553]:
%%system
cat WebExtract.txt | iconv -f latin1 -t utf-8 > WebExtract_cleaned.txt

[]

In [554]:
with open('WebExtract_cleaned.txt') as f:
    content = f.readlines()

recontent= [re.sub(r'","','";;"', caps) for caps in content] #substituting ;; for delimiting ,
#print(recontent[1])
recontent= [re.sub(r'\,',';', caps) for caps in recontent] #substituting ; for content ,
#print(recontent[1])
recontent= [re.sub(r'";;"','","', caps) for caps in recontent] #substituting , for content ;;
#print(recontent[1])

recontent2=[caps.split(',') for caps in recontent]
#print(recontent2[1])

recontent3=[]
for caps in recontent2:
    if len(caps)>11:
        caps[11]=(re.sub(r'""','"9999"',caps[11]))
    newcaps=",".join(caps)
    recontent3.append(newcaps)
    
print(recontent3[1])
recontent3= [re.sub(r'","',',', caps) for caps in recontent3] #substituting away ""
print(recontent3[1])
recontent3= [re.sub(r'^"','', caps) for caps in recontent3] #substituting ; for content ,
recontent3= [re.sub(r'".*$','', caps) for caps in recontent3] #substituting ; for content ,
renohead=recontent3[1:]
with open('we2', 'w') as f:
    f.writelines('%s' % l for l in renohead)
    
renohead=[]
for entries in recontent3[1:]:
    if len(entries)>34:
        renohead.append(entries)
renohead = renohead[:25930]+renohead[25931:]
with open('we2', 'w') as f:
    f.writelines('%s' % l for l in renohead)

"30075445","MORRIS PARK BAKE SHOP","2","1007      ","MORRIS PARK AVE                                   ","10462","7188924968","08","2014-03-03 00:00:00","D","10F","2","A","2014-03-03 00:00:00","2014-09-04 06:01:28.403000000"

"30075445,MORRIS PARK BAKE SHOP,2,1007      ,MORRIS PARK AVE                                   ,10462,7188924968,08,2014-03-03 00:00:00,D,10F,2,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000"



In [557]:
%%system
cat we2 | iconv -f latin1 -t utf-8 > we2f

[]

In [558]:
####TO EXPORT DATA FROM 'tasks' FILE TO SQL#####
################################################

import psycopg2
import sys


con = None
f = None

try:
     
    con = psycopg2.connect(database='vagrant', user='vagrant') 
    
    cur = con.cursor()
    f = open('we2f', 'r')
    cur.copy_from(f, 'grades', sep=",")                    
    con.commit()
   
except psycopg2.DatabaseError, e:
    
    if con:
        con.rollback()
    
    print 'Error %s' % e    
    sys.exit(1)

except IOError, e:    

    if con:
        con.rollback()

    print 'Error %s' % e   
    sys.exit(1)
    
finally:
    
    if con:
        con.close()

    if f:
        f.close()  


In [395]:
#TO COLLECT DATA FROM SQL DIRECTLY##
#####################################

con = None

try:
    con = psycopg2.connect(database='vagrant', user='vagrant')     
    cur = con.cursor()
    cur.execute("SELECT zipcode, AVG(score) from grades WHERE score<9999 GROUP BY zipcode")
    rows=cur.fetchall()
    
    ziplist=[]
    meanl=[]
    for row in rows:
        ziplist.append(row[0])
        meanl.append(row[1])

    cur.execute("SELECT zipcode, COUNT(score) from grades WHERE score<9999 GROUP BY zipcode")
    rows=cur.fetchall()
    
    inspections=[]
    for row in rows:
        inspections.append(row[1])

    cur.execute("SELECT zipcode, STDDEV(score) from grades WHERE score<9999 GROUP BY zipcode")
    rows=cur.fetchall()
    
    errorsl=[]
    for row in rows:
        errorsl.append(row[1])

    
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
    
finally:
    
    if con:
        con.close()

print(len(ziplist))

207


In [411]:
tupleslist=[]
for i in range(len(ziplist)):
    if inspections[i]>100:
        newentry=(str(ziplist[i]),float(meanl[i]),float(errorsl[i])/float(math.sqrt(inspections[i])),int(inspections[i]))
        tupleslist.append(newentry)
scorebyzipcodeout=(sorted(tupleslist,reverse=True))
print(len(scorebyzipcodeout))

output = open('../../miniprojects/questions/sql1.pickle','w')
pickle.dump(scorebyzipcodeout,output)
output.close()

183


## OUTPUTTING ZIPCODE FILE FOR CARTODB VISUALIZATION

I then used [CartoDB](http://cartodb.com/) to produce a map of average scores by zip code.

In [420]:
import csv

with open('mapping.csv', 'w') as csvfile:
    fieldnames = ['zip_code', 'score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(scorebyzipcodeout)):
        writer.writerow({'zip_code': scorebyzipcodeout[i][0], 'score': scorebyzipcodeout[i][1]})


## SORTING BY BOROUGH

Here, I extracted the same information (mean grade, stderr, number of inspections) for each of the city's five boroughs.

In [422]:
conn = psycopg2.connect("dbname='vagrant' user='vagrant'")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS boroughs")
cur.execute("CREATE TABLE boroughs (BORO INT, BORONAME TEXT)")
conn.commit()

In [424]:
conn = psycopg2.connect("dbname='vagrant' user='vagrant'")
cur = conn.cursor()
cur.execute("INSERT INTO boroughs VALUES (1, 'MANHATTAN')")
cur.execute("INSERT INTO boroughs VALUES (2, 'THE BRONX')")
cur.execute("INSERT INTO boroughs VALUES (3, 'BROOKLYN')")
cur.execute("INSERT INTO boroughs VALUES (4, 'QUEENS')")
cur.execute("INSERT INTO boroughs VALUES (5, 'STATEN ISLAND')")
conn.commit()

In [428]:
#TO COLLECT DATA FROM SQL DIRECTLY##
#####################################

con = None

sqlcmd = """
SELECT Boroughs.boroname, AVG(Grades.score), STDDEV(Grades.score), COUNT(Grades.score) as total
from Grades, Boroughs
WHERE Grades.boro = boroughs.boro and Grades.score<9999
GROUP BY Boroughs.boroname
ORDER BY total desc;
"""

try:
    con = psycopg2.connect(database='vagrant', user='vagrant')     
    cur = con.cursor()
    cur.execute(sqlcmd)
    rows=cur.fetchall()
    
    borolist=[]
    meanl=[]
    stdevl=[]
    inspections=[]
    for row in rows:
        borolist.append(row[0])
        meanl.append(row[1])
        stdevl.append(row[2])
        inspections.append(row[3])
    
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
    
finally:
    
    if con:
        con.close()

for i in range(len(borolist)):
    print(borolist[i],meanl[i],stdevl[i],inspections[i])

('MANHATTAN', Decimal('22.2356112023129885'), Decimal('15.0405292153990754'), 204065L)
('BROOKLYN', Decimal('22.1690296493647789'), Decimal('15.1678735108560759'), 116731L)
('QUEENS', Decimal('22.6888769757548508'), Decimal('15.5290453548602659'), 115652L)
('THE BRONX', Decimal('21.5671118128193321'), Decimal('15.1990054317136043'), 45603L)
('STATEN ISLAND', Decimal('20.9189491335941867'), Decimal('13.5851883826653510'), 16101L)


In [431]:
import math
import pickle

tupleslist2=[]
for i in range(len(borolist)):
    newentry=(str(borolist[i]),float(meanl[i]),float(stdevl[i])/float(math.sqrt(inspections[i])),int(inspections[i]))
    tupleslist2.append(newentry)
print(tupleslist2)

output = open('../../miniprojects/questions/sql2.pickle','w')
pickle.dump(tupleslist2,output)
output.close()

[('MANHATTAN', 22.235611202312988, 0.03329498747286129, 204065), ('BROOKLYN', 22.169029649364777, 0.044394748017859545, 116731), ('QUEENS', 22.68887697575485, 0.04566339364966828, 115652), ('THE BRONX', 21.56711181281933, 0.07117352299167724, 45603), ('STATEN ISLAND', 20.918949133594186, 0.10706295846600053, 16101)]


## SORTING BY CUISINE

I joined the 'cuisine' and 'boroughs' table, and extracted the summary statistics for the 75 cuisine types with at least 100 inspections.

In [2]:
conn = psycopg2.connect("dbname='vagrant' user='vagrant'")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS cuisine")
cur.execute("CREATE TABLE cuisine (CUISINECODE INT, CODEDESC TEXT)")
conn.commit()

In [3]:
%%system
cat Cuisine.txt | iconv -f latin1 -t utf-8 > Cuisine_cleaned.txt

[]

In [4]:
with open('Cuisine_cleaned.txt') as f:
    content = f.readlines()

recontent= [re.sub(r'","','";;"', caps) for caps in content] #substituting ;; for delimiting ,
#print(recontent[1])
recontent= [re.sub(r'\,',';', caps) for caps in recontent] #substituting ; for content ,
#print(recontent[1])
recontent= [re.sub(r'";;"','","', caps) for caps in recontent] #substituting , for content ;;
#print(recontent[1])
   
recontent= [re.sub(r'","',',', caps) for caps in recontent] #substituting away ""
recontent= [re.sub(r'^"','', caps) for caps in recontent] #substituting away beginning ""
recontent= [re.sub(r'".*$','', caps) for caps in recontent] #substituting away ending ""
renohead=recontent[1:]
with open('we2', 'w') as f:
    f.writelines('%s' % l for l in renohead)

print(renohead)

['02,African\n', '03,American \n', '05,Asian\n', '15,Cajun\n', '17,Caribbean\n', '20,Chinese\n', '30,Eastern European\n', '31,Egyptian\n', '35,French\n', '37,German\n', '38,Greek\n', '44,Indian\n', '45,Indonesian\n', '48,Italian\n', '49,Japanese\n', '50,Jewish/Kosher\n', '52,Korean\n', '53,Latin (Cuban; Dominican; Puerto Rican; South & Central American)\n', '54,Mediterranean\n', '55,Mexican\n', '56,Middle Eastern\n', '67,Russian\n', '76,Southwestern\n', '82,Thai\n', '99,Other\n', '01,Afghan\n', '04,Armenian\n', '06,Australian\n', '07,Bagels/Pretzels\n', '08,Bakery\n', '09,Bangladeshi\n', '10,Barbecue\n', '11,Basque\n', '12,Bottled beverages; including water; sodas; juices; etc.\n', '13,Brazilian\n', '14,Caf\xc3\xa9/Coffee/Tea\n', '16,Californian\n', '18,Chicken\n', '19,Chilean\n', '21,Chinese/Cuban\n', '22,Chinese/Japanese\n', '23,Continental\n', '24,Creole\n', '25,Creole/Cajun\n', '26,Czech\n', '27,Delicatessen\n', '28,Vietnamese/Cambodian/Malaysia\n', '29,Donuts\n', '32,English\n', '

In [5]:
%%system
cat we2 | iconv -f latin1 -t utf-8 > we2f

[]

In [6]:
####TO EXPORT DATA FROM 'tasks' FILE TO SQL#####
################################################

import psycopg2
import sys


con = None
f = None

try:
     
    con = psycopg2.connect(database='vagrant', user='vagrant') 
    
    cur = con.cursor()
    f = open('we2f', 'r')
    cur.copy_from(f, 'cuisine', sep=",")                    
    con.commit()
   
except psycopg2.DatabaseError, e:
    
    if con:
        con.rollback()
    
    print 'Error %s' % e    
    sys.exit(1)

except IOError, e:    

    if con:
        con.rollback()

    print 'Error %s' % e   
    sys.exit(1)
    
finally:
    
    if con:
        con.close()

    if f:
        f.close()  


In [442]:
#TO COLLECT DATA FROM SQL DIRECTLY##
#####################################

con = None

sqlcmd = """
SELECT Cuisine.codedesc, AVG(Grades.score) as avgscore, STDDEV(Grades.score), COUNT(Grades.score) as totalc
from Grades, Cuisine
WHERE Grades.cuisinecode = Cuisine.cuisinecode and Grades.score<9999
GROUP BY Cuisine.cuisinedesc
ORDER BY totalc desc;
"""

try:
    con = psycopg2.connect(database='vagrant', user='vagrant')     
    cur = con.cursor()
    cur.execute(sqlcmd)
    rows=cur.fetchall()
    
    cuisinelist=[]
    meanl=[]
    stdevl=[]
    inspections=[]
    for row in rows:
        cuisinelist.append(row[0])
        meanl.append(row[1])
        stdevl.append(row[2])
        inspections.append(row[3])
    
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
    
finally:
    
    if con:
        con.close()

for i in range(len(cuisinelist)):
    print(cuisinelist[i],meanl[i],stdevl[i],inspections[i])

('American ', Decimal('21.3551313249979022'), Decimal('14.4033176318749135'), 119170L)
('Chinese', Decimal('24.9211521577632201'), Decimal('16.5986714155487363'), 59228L)
('Latin (Cuban; Dominican; Puerto Rican; South & Central American)', Decimal('24.4995562726906010'), Decimal('16.1821486376875136'), 24790L)
('Pizza', Decimal('21.3703342278604538'), Decimal('14.3912077153683614'), 24594L)
('Italian', Decimal('22.1795240639513378'), Decimal('14.5315194348504420'), 24331L)
('Japanese', Decimal('23.8085913181277944'), Decimal('15.2168623919287750'), 17669L)
('Mexican', Decimal('23.2600883990586074'), Decimal('14.8921467747433057'), 17421L)
('Bakery', Decimal('23.0654289950364211'), Decimal('16.1699422170539853'), 15513L)
('Caribbean', Decimal('23.0925961849337213'), Decimal('15.7306763233733266'), 15465L)
('Caf\xc3\x83\xc2\xa9/Coffee/Tea', Decimal('17.1263705369693562'), Decimal('12.3408424825936259'), 14228L)
('Spanish', Decimal('23.4607191479138304'), Decimal('15.5888746609285937'), 1

In [443]:
import math
import pickle

tupleslist3=[]
for i in range(75):
    newentry=(str(cuisinelist[i]),float(meanl[i]),float(stdevl[i])/float(math.sqrt(inspections[i])),int(inspections[i]))
    tupleslist3.append(newentry)
print(tupleslist3[0:75])

output = open('../../miniprojects/questions/sql3.pickle','w')
pickle.dump(tupleslist3,output)
output.close()

[('American ', 21.355131324997902, 0.041723340151607224, 119170), ('Chinese', 24.92115215776322, 0.06820399186759622, 59228), ('Latin (Cuban; Dominican; Puerto Rican; South & Central American)', 24.4995562726906, 0.1027774699594768, 24790), ('Pizza', 21.370334227860454, 0.09176618080188305, 24594), ('Italian', 22.179524063951337, 0.09316033631827197, 24331), ('Japanese', 23.808591318127796, 0.11447723411350752, 17669), ('Mexican', 23.260088399058606, 0.11282900740718464, 17421), ('Bakery', 23.06542899503642, 0.12982566974763338, 15513), ('Caribbean', 23.092596184933722, 0.12649473014067292, 15465), ('Caf\xc3\x83\xc2\xa9/Coffee/Tea', 17.126370536969358, 0.10346009689186739, 14228), ('Spanish', 23.46071914791383, 0.1395036945301105, 12487), ('Pizza/Italian', 22.192953184807145, 0.13975090047795347, 10189), ('Delicatessen', 24.519510080208107, 0.16235492849437935, 9226), ('Indian', 25.093488725614392, 0.2024034646705623, 7894), ('Hamburgers', 17.513092520478043, 0.13242022119859062, 7447)

## CALCULATING VIOLATIONS BY TYPE OF RESTAURANT

Now, to answer a more complicated question -- which cuisines tend to have a disproportionate number of what which violations? Some notes:

(1) More popular cuisine categories will tend to have more violations just because they represent more restaurants. Similarly, some violations are more common.  For example, knowing that "Equipment not easily movable or sealed to floor" is a common violation for Chinese restuarants is not particularly helpful when it is a common violation for all restaurants. The right quantity is to look at is the conditional probability of a specific type of violation given a specific cuisine type and divide it by the unconditional probability of the violation for the entire population.

(2) The definition of a violation changes with time.  For example, 10A can mean two different things "Toilet facility not maintained ..." or "Vermin or other live animal present ..." when things were prior to 2003.

(3) The ratios don't mean much when the number of violations of a given type and for a specific category are not large, so I filtered them out (using 100 as a somewhat arbitrary threshold).

In [15]:
conn = psycopg2.connect("dbname='vagrant' user='vagrant'")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS violations")
cur.execute("CREATE TABLE violations (STARTDATE timestamp, ENDDATE timestamp, CRITICALFLAG varchar(1), VIOLATIONCODE varchar(3),VIOLATIONDESC varchar(512))")
conn.commit()            

In [509]:
%%system
cat Violation.txt | iconv -f latin1 -t utf-8 > Violation_cleaned.txt

[]

In [10]:
with open('Violation.txt') as f:
    content = f.readlines()
recontent= [re.sub(r'","','";;"', caps) for caps in content] #substituting ;; for delimiting ,
print(recontent[1])
recontent= [re.sub(r'\,',';', caps) for caps in recontent] #substituting ; for content ,
print(recontent[1])
recontent= [re.sub(r'";;"','","', caps) for caps in recontent] #substituting , for delimiting ;;
recontent= [re.sub(r'","',',', caps) for caps in recontent] #getting rid of middle quotes
recontent= [re.sub(r'^"','', caps) for caps in recontent] #getting rid of beginning quote
renohead=recontent[1:]
print(renohead[0:20])
print(len(renohead))

"1901-01-01 00:00:00";;"2003-03-23 00:00:00";;"Y";;"01A";;"Current valid <a    onmouseover="ShowContent('P2','01A'); return true;"     href="javascript:ShowContent('P2','01A')">permit</A> , registration or other authorization to operate establishment not available."

"1901-01-01 00:00:00";;"2003-03-23 00:00:00";;"Y";;"01A";;"Current valid <a    onmouseover="ShowContent('P2';'01A'); return true;"     href="javascript:ShowContent('P2';'01A')">permit</A> ; registration or other authorization to operate establishment not available."

['1901-01-01 00:00:00,2003-03-23 00:00:00,Y,01A,Current valid <a    onmouseover="ShowContent(\'P2\';\'01A\'); return true;"     href="javascript:ShowContent(\'P2\';\'01A\')">permit</A> ; registration or other authorization to operate establishment not available."\r\n', '2003-03-24 00:00:00,2005-02-17 00:00:00,Y,01A,Current valid <a    onmouseover="ShowContent(\'P2\';\'01A\'); return true;"     href="javascript:ShowContent(\'P2\';\'01A\')">permit</A> ; registra

In [11]:
with open('violations2', 'w') as f:
    f.writelines('%s' % l for l in renohead)

In [12]:
%%system
cat violations2 | iconv -f latin1 -t utf-8 > violations2f

[]

In [13]:
!cat violations2f

In [16]:
####TO EXPORT DATA FROM outside FILE TO SQL#####
################################################

import psycopg2
import sys


con = None
f = None

try:
     
    con = psycopg2.connect(database='vagrant', user='vagrant') 
    
    cur = con.cursor()
    f = open('violations2f', 'r')
    cur.copy_from(f, 'Violations', sep=",")                    
    con.commit()
   
except psycopg2.DatabaseError, e:
    
    if con:
        con.rollback()
    
    print 'Error %s' % e    
    sys.exit(1)

except IOError, e:    

    if con:
        con.rollback()

    print 'Error %s' % e   
    sys.exit(1)
    
finally:
    
    if con:
        con.close()

    if f:
        f.close()  


In [17]:
conn = psycopg2.connect("dbname='vagrant' user='vagrant'")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS violationsCategorized")
cur.execute("SELECT VIOLATIONDESC, COUNT(*), CODEDESC INTO violationsCategorized FROM (SELECT CUISINECODE, VIOLATIONDESC FROM grades LEFT JOIN violations ON VIOLCODE=VIOLATIONCODE WHERE INSPDATE BETWEEN STARTDATE AND ENDDATE) AS foo1 LEFT JOIN cuisine ON foo1.CUISINECODE=cuisine.CUISINECODE GROUP BY VIOLATIONDESC, CODEDESC")
conn.commit()

In [71]:
conn = psycopg2.connect("dbname='vagrant' user='vagrant'")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS t2")
cur.execute("CREATE TABLE t2 AS (SELECT violationdesc, SUM(violationsCategorized.count) AS FREQ from violationscategorized GROUP BY violationdesc)")
cur.execute("DROP TABLE IF EXISTS violationscategorized2")
cur.execute("CREATE TABLE violationscategorized2 AS (SELECT violationscategorized.codedesc, violationscategorized.violationdesc, violationscategorized.count, t2.freq*1.0/(select sum(freq) from t2) AS vprob from violationscategorized INNER JOIN t2 on violationscategorized.violationdesc=t2.violationdesc)")
conn.commit()

In [78]:
#TO COLLECT DATA FROM SQL DIRECTLY##
#####################################

con = None

sqlcmd2="""
Select  s1.codedesc, s1.violationdesc, (s1.count*1.0/s2.Count2s)/s1.vprob as freq1, s2.Count2s, s1.vprob, s1.count
From    (
        Select codedesc, violationdesc, count, vprob
        From   violationscategorized2
        ) As s1
        Inner Join (
            Select codedesc, Count(*) As Count2, Sum(count) As Count2s
            From   violationscategorized2
            Group By codedesc
            ) As s2
            On s1.codedesc = s2.codedesc
            
ORDER BY freq1 desc, s1.count desc;
"""

try:
    con = psycopg2.connect(database='vagrant', user='vagrant')     
    cur = con.cursor()
    cur.execute(sqlcmd2)
    rows=cur.fetchall()
    
    cuisinel=[]
    viol=[]
    freql=[]
    countb=[]
    probl=[]
    countsl=[]
    for row in rows:
        cuisinel.append(row[0])
        viol.append(row[1])
        freql.append(row[2])
        countb.append(row[3])
        probl.append(row[4])
        countsl.append(row[5])
    
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
    
finally:
    
    if con:
        con.close()
        
xlen=len(cuisinel)
for i in range(xlen):
    print(cuisinel[i],viol[i],float(freql[i]),int(countsl[i]))

('Cajun', 'Specific caloric content or range thereof not posted on menus; menu boards or food tags for each menu item offered as a combination meal with multiple options that are listed as single items."', 290.8100558659218, 1)
('Portuguese', 'Food not labeled in accordance with HACCP plan."', 168.27218361079684, 2)
('Egyptian', 'Out-of package sale of tobacco products observed."', 135.9848484848485, 1)
('Middle Eastern', 'Out-of package sale of tobacco products observed."', 99.98765194961996, 7)
('Fruits/Vegetables', 'Food service operation occurring in room used as living or sleeping quarters."', 91.95371842430666, 1)
('Californian', 'Food not cooled by an approved method whereby the internal product temperature is reduced from 140\xc2\xba F to 70\xc2\xba F or less within 2 hours; and from 70\xc2\xba F to 41\xc2\xba F or less within 4 additional hours."', 82.36550632911393, 1)
('Afghan', 'Unprotected food re-served."', 75.44202898550725, 1)
('Hamburgers', 'Evidence of flying insects 

In [81]:
recuisine= [re.sub(r'Caf\xc3\x83\xc2\xa9/Coffee/Tea','Caf?/Coffee/Tea', vmessages) for vmessages in cuisinel] #substituting away ""
reviolist= [re.sub(r'\xc2\x93Choking first aid\xc2\x94 poster not posted. \xc2\x93Alcohol and pregnancy\xc2\x94 warning','?Choking first aid? poster not posted. ?Alcohol and pregnancy? warning', vmessages) for vmessages in viol] #substituting away ""
reviolist= [re.sub(r'140\xc2\xba F to 70\xc2\xba F or less within 2 hours; and from 70\xc2\xba F to 41\xc2\xba F','140? F to 70? F or less within 2 hours; and from 70? F to 41? F', vmessages) for vmessages in reviolist] #substituting away ""
reviolist= [re.sub(r'"$','', vmessages) for vmessages in reviolist] #substituting away ""

tupleslist4=[]
countv=0
for i in range(len(violist)):
    if int(countsl[i])>100:
        countv=countv+1
        if countv<21:
            newentry =((recuisine[i],reviolist[i]),float(freql[i]),int(countsl[i]))
            tupleslist4.append(newentry)
tupleslist4s=sorted(tupleslist4,key=lambda x: x[2])
print(tupleslist4)
output = open('../../miniprojects/questions/sql4.pickle','w')
pickle.dump(tupleslist4,output)
output.close()

[(('Japanese', 'Food worker does not use proper utensil to eliminate bare hand contact with food that will not receive adequate additional heat treatment.'), 3.2451745268476633, 541), (('Caf?/Coffee/Tea', '?Choking first aid? poster not posted. ?Alcohol and pregnancy? warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric); latex gloves; sign not posted. Inspection report sign not posted.'), 3.152037031558518, 175), (('Juice; Smoothies; Fruit Salads', 'Food Protection Certificate not held by supervisor of food operations.'), 3.06821797767629, 143), (('Donuts', 'Accurate thermometer not provided in refrigerated or hot holding equipment.'), 3.0380292493733503, 130), (('Ice Cream; Gelato; Yogurt; Ices', 'Food Protection Certificate not held by supervisor of food operations.'), 2.9562605563761983, 193), (('Thai', 'Thawing procedures improper.'), 2.632134733815779, 151), (('Irish', 'Raw; cooked or prepared food is adulterated; contaminated; cros